# SPAM mail 구분하기

In [ ]:
전처리
로지스틱
스팸구분

In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving spam.csv to spam.csv


In [22]:
import numpy as np
import pandas as pd

df = pd.read_csv(filename, encoding='latin1')
df.head(2)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN


In [25]:
del df['Unnamed: 3']
del df['Unnamed: 4']
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [26]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [90]:
import re
df['clean_doc'] = df.clean_doc.str.replace('[^A-Za-z]', ' ')
df.tail(3)

,v1,v2,clean_doc
5569,ham,"Pity, * was in mood for that. So...any other s...",Pity was in mood for that Soany other suggest...
5570,ham,The guy did some bitching but I acted like i'd...,The guy did some bitching but I acted like id ...
5571,ham,Rofl. Its true to its name,Rofl Its true to its name


In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.clean_doc, df.v1, 
    test_size=1/3,
    stratify=df.v1, random_state=2021
    )
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((3714,), (1858,), (3714,), (1858,))

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
cvect1 = CountVectorizer(stop_words='english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)
X_train_cv1.shape, X_test_cv1.shape

((3714, 6494), (1858, 6494))

In [99]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((3714, 25898), (1858, 25898))

In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect1 = TfidfVectorizer(stop_words='english')
tvect1.fit(X_train)
X_train_tv1 = tvect1.transform(X_train)
X_test_tv1 = tvect1.transform(X_test)
X_train_tv1.shape, X_test_tv1.shape

((3714, 6494), (1858, 6494))

In [100]:
tvect2 = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)
X_train_tv2.shape, X_test_tv2.shape

((3714, 25898), (1858, 25898))

In [103]:
from sklearn.linear_model import LogisticRegression

In [104]:
lr1 = LogisticRegression(max_iter=500)
%time lr1.fit(X_train_cv1, y_train)

CPU times: user 40.6 ms, sys: 0 ns, total: 40.6 ms
Wall time: 48.5 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
lr2 = LogisticRegression(max_iter=500)
%time lr2.fit(X_train_cv2, y_train)

CPU times: user 122 ms, sys: 71.1 ms, total: 193 ms
Wall time: 113 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [106]:
lr3 = LogisticRegression(max_iter=500)
%time lr3.fit(X_train_tv1, y_train)

CPU times: user 49.3 ms, sys: 1.01 ms, total: 50.3 ms
Wall time: 52 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [107]:
lr4 = LogisticRegression(max_iter=500)
%time lr4.fit(X_train_tv2, y_train)

CPU times: user 138 ms, sys: 203 ms, total: 341 ms
Wall time: 185 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [108]:
pred1 = lr1.predict(X_test_cv1)

In [109]:
pred2 = lr2.predict(X_test_cv2)

In [110]:
pred3 = lr3.predict(X_test_tv1)

In [111]:
pred4 = lr4.predict(X_test_tv2)

In [112]:
from sklearn.metrics import accuracy_score

In [113]:
score1 = accuracy_score(y_test, pred1)
score1

0.9736275565123789

In [114]:
score2 = accuracy_score(y_test, pred2)
score2

0.9714747039827771

In [115]:
score3 = accuracy_score(y_test, pred3)
score3

0.9547900968783638

In [116]:
score4 = accuracy_score(y_test, pred4)
score4

0.9391819160387513

# Overview + 감독 + 주연배우 의 Cosine_similarity 구하기

In [117]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving credits.csv to credits.csv


In [121]:
from google.colab import files
uploaded = files.upload()
filename2 = list(uploaded.keys())[0]

Saving movies_metadata.csv to movies_metadata.csv


In [ ]:
data = np.loadtxt(filename, delimiter=',')

In [123]:
import pandas as pd

info = pd.read_csv(filename, low_memory=False)
movie = pd.read_csv(filename2, low_memory=False)


In [141]:
df = movie[['id', 'title', 'overview']]
df[df.id =='1997-08-20']


AttributeError: ignored

In [140]:
df2= df2.merge(df1,on='id')

TypeError: ignored

In [134]:
df1.head(5)

AttributeError: ignored

In [119]:
df['cast'] = df['cast'].apply(lambda x: ' '.join([i['name'].replace(' ', '') for i in eval(x)]))
df['crew'] = df['crew'].apply(lambda x: ' '.join([i['name'].replace(' ', '') for i in eval(x)]))

In [120]:
df

,cast,crew,id
0,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter JossWhedon AndrewStanton JoelCohe...,862
1,RobinWilliams JonathanHyde KirstenDunst Bradle...,LarryJ.Franco JonathanHensleigh JamesHorner Jo...,8844
2,WalterMatthau JackLemmon Ann-Margret SophiaLor...,HowardDeutch MarkStevenJohnson MarkStevenJohns...,15602
3,WhitneyHouston AngelaBassett LorettaDevine Lel...,ForestWhitaker RonaldBass RonaldBass EzraSwerd...,31357
4,SteveMartin DianeKeaton MartinShort KimberlyWi...,AlanSilvestri ElliotDavis NancyMeyers NancyMey...,11862
...,...,...,...
45471,LeilaHatami KouroshTahami ElhamKorda,HamidNematollah HamidNematollah FarshadMohamma...,439050
45472,AngelAquino PerryDizon HazelOrencio JoelTorre ...,LavDiaz LavDiaz DantePerez LavDiaz LavDiaz Lav...,111109
45473,ErikaEleniak AdamBaldwin JulieduPage JamesRema...,MarkL.Lester C.CourtneyJoyner JeffreyGoldenber...,67758
45474,IwanMosschuchin NathalieLissenko PavelPavlov A...,YakovProtazanov JosephN.Ermolieff,227506
